In [6]:
import pandas as pd
import psycopg2

df = pd.read_csv('data_upd.csv')

C:\Users\lavan\AppData\Local\Temp\ipykernel_8368\146285214.py:4: DtypeWarning: Columns (76,82,83,115,116,120,121,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_upd.csv')


In [7]:

df['Birthdate'] = pd.to_datetime(df['Birthdate'])
df['Date_of_death'] = pd.to_datetime(df['Date_of_death'])


In [8]:
df.dtypes

I                        int64
ID                       int64
NAME                    object
COUNTRY                 object
Full name               object
                        ...   
BowlingRank_Test       float64
AllRounderRank_Test    float64
BattingRank_T20        float64
BowlingRank_T20        float64
AllRounderRank_T20     float64
Length: 146, dtype: object

In [9]:

df['Age'] = df['Age'].fillna(0)
df['Age'] = df['Age'].astype(int)
df['NAME'] = df['NAME'].fillna('Unknown')
df['NAME'] = df['NAME'].astype(str)
df['Full name'] = df['Full name'].fillna('Unknown')
df['Full name'] = df['Full name'].astype(str)
df['Birthplace'] = df['Birthplace'].fillna('Unknown')
df['Birthplace'] = df['Birthplace'].astype(str)
df['Other'] = df['Other'].astype(str)
df['AWARDS'] = df['AWARDS'].astype(str)
df['COUNTRY'] = df['COUNTRY'].fillna('Unknown')
df['COUNTRY'] = df['COUNTRY'].astype(str)
df['Batting style'] = df['Batting style'].fillna('Unknown')
df['Batting style'] = df['Batting style'].astype(str)
df['Bowling style'] = df['Bowling style'].fillna('Unknown')
df['Bowling style'] = df['Bowling style'].astype(str)



## Player Table

In [10]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)

cursor = conn.cursor()
conn.commit()


# create_table_query = """
# CREATE TABLE IF NOT EXISTS Player (
#   PlayerID SERIAL PRIMARY KEY,
#   Name VARCHAR NOT NULL,
#   FullName VARCHAR,
#   BirthDate DATE NULL,
#   DeathDate DATE NULL,
#   Age INTEGER,
#   IsAlive BOOLEAN DEFAULT TRUE
# )
# """
create_table_query = """
CREATE TABLE IF NOT EXISTS Player (
  PlayerID SERIAL PRIMARY KEY,
  Name VARCHAR NOT NULL,
  FullName VARCHAR,
  BirthDate DATE NULL,
  DeathDate DATE NULL,
  Age INTEGER,
  IsAlive BOOLEAN DEFAULT TRUE
);"""

cursor.execute(create_table_query)
conn.commit()



for index, row in df.iterrows():
    insert_query = f"""
    INSERT INTO Player (Name, FullName, BirthDate, DeathDate, Age, IsAlive)
    VALUES (
        '{row['NAME'].replace("'","''")}', 
        '{row['Full name'].replace("'","''")}', 
        CASE 
            WHEN '{row['Birthdate']}' = 'NaT' THEN NULL 
            ELSE to_date('{row['Birthdate']}', 'YYYY-MM-DD')
        END, 
        CASE 
            WHEN '{row['Date_of_death']}' = 'NaT' THEN NULL 
            ELSE to_date('{row['Date_of_death']}', 'YYYY-MM-DD')
        END, 
        '{row['Age']}',
        CASE 
            WHEN '{row['Died']}' = 'Dead' OR '{row['Date_of_death']}' is NOT NULL THEN FALSE
            ELSE TRUE
        END
    )
    """
    cursor.execute(insert_query)
    conn.commit()


cursor.close()
conn.close()



In [11]:
#Players dictionary
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()

cursor.execute("SELECT playerid,FullName FROM Player")
players = cursor.fetchall()
players_dict = {}
for player in players:
    players_dict[player[1]] = player[0]

cursor.close()
conn.close()


## Country Table

In [12]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS Country (
    CountryId SERIAL PRIMARY KEY,
    CountryName TEXT UNIQUE NOT NULL
);
"""
cursor.execute(create_table_query)
conn.commit()

for index, row in df.iterrows():
    insert_query = f"""
    INSERT INTO Country (CountryName)
    VALUES ('{row['COUNTRY'].replace("'","")}')
    ON CONFLICT DO NOTHING
    """
    cursor.execute(insert_query)
    conn.commit()

cursor.close()
conn.close()


In [13]:
#country dictionary
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()
conn.commit()
cursor.execute("SELECT * FROM Country")
countries = cursor.fetchall()
countries_dict = {}
for country in countries:
    countries_dict[country[1]] = country[0]

cursor.close()
conn.close()

## PlayerTeam table

In [14]:
# conn = psycopg2.connect(
#     host="localhost",
#     database="ESPNCRICINFO",
#     user="karthik",
#     password="karthik"
# )

# cursor = conn.cursor()
# conn.commit()

# create_table_query = """
# CREATE TABLE IF NOT EXISTS PlayerTeam (
#     PlayerID INTEGER REFERENCES Player(PlayerID),
#     Teams VARCHAR(1000))"""

# cursor.execute(create_table_query)
# conn.commit()

# for index, row in df.iterrows():
#     player_id = players_dict[row['NAME']]
#     teams = row['Major teams']
#     if pd.isnull(teams):
#         teams = ''
#     else:
#         teams = teams.replace("'", "")
#     insert_query = f"""
#     INSERT INTO PlayerTeam(PlayerID, Teams)
#     VALUES('{player_id}' , '{teams}')
#     """
#     cursor.execute(insert_query)
#     conn.commit()

# cursor.close()
# conn.close()


## PlayerDetails Table

In [15]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS PlayerDetails (
    PlayerID INTEGER REFERENCES Player(PlayerID),
    Birthplace VARCHAR(250),
    COUNTRYID INTEGER REFERENCES Country(CountryID),
    BattingStyle VARCHAR(150),
    BowlingStyle VARCHAR(250),
    Height INT,
    Weight INT,
    Teams VARCHAR(1000),
    OtherRoles VARCHAR(250),
    Awards VARCHAR(4000))
"""

cursor.execute(create_table_query)
conn.commit()

df['Major teams'] = df['Major teams'].fillna('')

for index, row in df.iterrows():
    insert_query = f"""
    INSERT INTO PlayerDetails(PlayerID, Birthplace, COUNTRYID, BattingStyle, BowlingStyle, Height, Weight, Teams, OtherRoles, Awards)
    VALUES('{players_dict[row['Full name']]}' , '{row['Birthplace'].replace("'","")}' , '{countries_dict[row['COUNTRY']]}' , '{row['Batting style'].replace("'","")}' , '{row['Bowling style'].replace("'","")}',
    '{row['Height']}' , '{row['Weight']}', '{row['Major teams'].replace("'","")}' , '{row['Other'].replace("'","")}' , '{row['AWARDS'].replace("'","")}')
    """
    cursor.execute(insert_query)
    conn.commit()

cursor.close()
conn.close()


## Format Table

In [16]:

conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()
conn.commit()
create_table_query = """
    CREATE TABLE IF NOT EXISTS Format (
        FormatID SERIAL PRIMARY KEY,
        FormatName TEXT UNIQUE NOT NULL,
        Overs INT NOT NULL
    )
"""

cursor.execute(create_table_query)


values = [
    ("TEST", 450),
    ("ODI", 50),
    ("T20", 20),
    ("FirstClass", 450)
]

insert_query = f"Insert into Format (FormatName, Overs) values (%s, %s)"

cursor.executemany(insert_query, values)

conn.commit()

cursor.close()
conn.close()




In [17]:
#Format dictionary
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()

cursor.execute("SELECT formatid,formatname FROM Format")
formats = cursor.fetchall()
formats_dict = {}
for format in formats:
    formats_dict[format[1]] = format[0]

print(formats_dict)

cursor.close()
conn.close()


{'TEST': 1, 'ODI': 2, 'T20': 3, 'FirstClass': 4}


## Batting Stats table

In [18]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)


cursor = conn.cursor()
conn.commit()

create_query = '''CREATE TABLE IF NOT EXISTS BattingStats (
                    BattingStatsID SERIAL PRIMARY KEY,
                    FormatId INT REFERENCES Format(FormatID),
                    PlayerId INT REFERENCES Player(PlayerId),
                    Matches INT,
                    Innings INT,
                    NotOuts INT,
                    Runs INT,
                    HighestScore VARCHAR(50),
                    Average FLOAT,
                    BallsFaced INT,
                    StrikeRate FLOAT,
                    Centuries INT,
                    Fifties INT,
                    Fours INT,
                    Sixers INT,
                    CHECK (Matches >= 0),
                    CHECK (Innings >= 0),
                    CHECK (NotOuts >= 0),
                    CHECK (Runs >= 0),
                    CHECK (BallsFaced >= 0),
                    CHECK (Centuries >= 0),
                    CHECK (Fifties >= 0),
                    CHECK (Fours >= 0),
                    CHECK (Sixers >= 0)                    
                );'''

cursor.execute(create_query)


df['BATTING_Tests_Mat'] = df['BATTING_Tests_Mat'].fillna(0).astype(int)
df['BATTING_Tests_Inns'] = df['BATTING_Tests_Inns'].fillna(0).astype(int)
df['BATTING_Tests_NO'] = df['BATTING_Tests_NO'].fillna(0).astype(int)
df['BATTING_Tests_Runs'] = df['BATTING_Tests_Runs'].fillna(0).astype(int)
df['BATTING_Tests_Ave'] = df['BATTING_Tests_Ave'].fillna(0).astype(float)
df['BATTING_Tests_BF'] = df['BATTING_Tests_BF'].fillna(0).astype(int)
df['BATTING_Tests_SR'] = df['BATTING_Tests_SR'].fillna(0).astype(float)
df['BATTING_Tests_100'] = df['BATTING_Tests_100'].fillna(0).astype(int)
df['BATTING_Tests_50'] = df['BATTING_Tests_50'].fillna(0).astype(int)
df['BATTING_Tests_4s'] = df['BATTING_Tests_4s'].fillna(0).astype(int)
df['BATTING_Tests_6s'] = df['BATTING_Tests_6s'].fillna(0).astype(int)


df['BATTING_ODIs_Mat'] = df['BATTING_ODIs_Mat'].fillna(0).astype(int)
df['BATTING_ODIs_Inns'] = df['BATTING_ODIs_Inns'].fillna(0).astype(int)
df['BATTING_ODIs_NO'] = df['BATTING_ODIs_NO'].fillna(0).astype(int)
df['BATTING_ODIs_Runs'] = df['BATTING_ODIs_Runs'].fillna(0).astype(int)
df['BATTING_ODIs_Ave'] = df['BATTING_ODIs_Ave'].fillna(0).astype(float)
df['BATTING_ODIs_BF'] = df['BATTING_ODIs_BF'].fillna(0).astype(int)
df['BATTING_ODIs_SR'] = df['BATTING_ODIs_SR'].fillna(0).astype(float)
df['BATTING_ODIs_100'] = df['BATTING_ODIs_100'].fillna(0).astype(int)
df['BATTING_ODIs_50'] = df['BATTING_ODIs_50'].fillna(0).astype(int)
df['BATTING_ODIs_4s'] = df['BATTING_ODIs_4s'].fillna(0).astype(int)
df['BATTING_ODIs_6s'] = df['BATTING_ODIs_6s'].fillna(0).astype(int)


df['BATTING_T20Is_Mat'] = df['BATTING_T20Is_Mat'].fillna(0).astype(int)
df['BATTING_T20Is_Inns'] = df['BATTING_T20Is_Inns'].fillna(0).astype(int)
df['BATTING_T20Is_NO'] = df['BATTING_T20Is_NO'].fillna(0).astype(int)
df['BATTING_T20Is_Runs'] = df['BATTING_T20Is_Runs'].fillna(0).astype(int)
df['BATTING_T20Is_Ave'] = df['BATTING_T20Is_Ave'].fillna(0).astype(float)
df['BATTING_T20Is_BF'] = df['BATTING_T20Is_BF'].fillna(0).astype(int)
df['BATTING_T20Is_SR'] = df['BATTING_T20Is_SR'].fillna(0).astype(float)
df['BATTING_T20Is_100'] = df['BATTING_T20Is_100'].fillna(0).astype(int)
df['BATTING_T20Is_50'] = df['BATTING_T20Is_50'].fillna(0).astype(int)
df['BATTING_T20Is_4s'] = df['BATTING_T20Is_4s'].fillna(0).astype(int)
df['BATTING_T20Is_6s'] = df['BATTING_T20Is_6s'].fillna(0).astype(int)

df['BATTING_First-class_Mat'] = df['BATTING_First-class_Mat'].fillna(0).astype(int)
df['BATTING_First-class_Inns'] = df['BATTING_First-class_Inns'].fillna(0).astype(int)
df['BATTING_First-class_NO'] = df['BATTING_First-class_NO'].fillna(0).astype(int)
df['BATTING_First-class_Runs'] = df['BATTING_First-class_Runs'].fillna(0).astype(int)
df['BATTING_First-class_Ave'] = df['BATTING_First-class_Ave'].fillna(0).astype(float)
df['BATTING_First-class_BF'] = df['BATTING_First-class_BF'].fillna(0).astype(int)
df['BATTING_First-class_SR'] = df['BATTING_First-class_SR'].fillna(0).astype(float)
df['BATTING_First-class_100'] = df['BATTING_First-class_100'].fillna(0).astype(int)
df['BATTING_First-class_50'] = df['BATTING_First-class_50'].fillna(0).astype(int)
df['BATTING_First-class_4s'] = df['BATTING_First-class_4s'].fillna(0).astype(int)
df['BATTING_First-class_6s'] = df['BATTING_First-class_6s'].fillna(0).astype(int)


for index, row in df.iterrows():
    if row['BATTING_Tests_Mat'] > 0:
        insert_query = f"""
        INSERT INTO BattingStats(FormatId, PlayerId, Matches, Innings, NotOuts, Runs, HighestScore, Average, BallsFaced, StrikeRate, Centuries, Fifties, Fours, Sixers)
        VALUES('{formats_dict['TEST']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_Tests_Mat']}' , '{row['BATTING_Tests_Inns']}' , '{row['BATTING_Tests_NO']}' , '{row['BATTING_Tests_Runs']}' , '{row['BATTING_Tests_HS']}' , '{row['BATTING_Tests_Ave']}' , '{row['BATTING_Tests_BF']}' , '{row['BATTING_Tests_SR']}' , '{row['BATTING_Tests_100']}' , '{row['BATTING_Tests_50']}' , '{row['BATTING_Tests_4s']}' , '{row['BATTING_Tests_6s']}')
        """
        cursor.execute(insert_query)
        conn.commit()

    if row['BATTING_ODIs_Mat'] > 0:
        insert_query = f"""
        INSERT INTO BattingStats(FormatId, PlayerId, Matches, Innings, NotOuts, Runs, HighestScore, Average, BallsFaced, StrikeRate, Centuries, Fifties, Fours, Sixers)
        VALUES('{formats_dict['ODI']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_ODIs_Mat']}' , '{row['BATTING_ODIs_Inns']}' , '{row['BATTING_ODIs_NO']}' , '{row['BATTING_ODIs_Runs']}' , '{row['BATTING_ODIs_HS']}' , '{row['BATTING_ODIs_Ave']}' , '{row['BATTING_ODIs_BF']}' , '{row['BATTING_ODIs_SR']}' , '{row['BATTING_ODIs_100']}' , '{row['BATTING_ODIs_50']}' , '{row['BATTING_ODIs_4s']}' , '{row['BATTING_ODIs_6s']}' )
        """
        cursor.execute(insert_query)
        conn.commit()

    if row['BATTING_T20Is_Mat'] > 0:
        insert_query = f"""
        INSERT INTO BattingStats(FormatId, PlayerId, Matches, Innings, NotOuts, Runs, HighestScore, Average, BallsFaced, StrikeRate, Centuries, Fifties, Fours, Sixers)
        VALUES('{formats_dict['T20']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_T20Is_Mat']}' , '{row['BATTING_T20Is_Inns']}' , '{row['BATTING_T20Is_NO']}' , '{row['BATTING_T20Is_Runs']}' , '{row['BATTING_T20Is_HS']}' , '{row['BATTING_T20Is_Ave']}' , '{row['BATTING_T20Is_BF']}' , '{row['BATTING_T20Is_SR']}' , '{row['BATTING_T20Is_100']}' , '{row['BATTING_T20Is_50']}' , '{row['BATTING_T20Is_4s']}' , '{row['BATTING_T20Is_6s']}' )
        """
        cursor.execute(insert_query)
        conn.commit()

    if row['BATTING_First-class_Mat'] > 0:
        insert_query = f"""
        INSERT INTO BattingStats(FormatId, PlayerId, Matches, Innings, NotOuts, Runs, HighestScore, Average, BallsFaced, StrikeRate, Centuries, Fifties, Fours, Sixers)
        VALUES('{formats_dict['FirstClass']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_First-class_Mat']}' , '{row['BATTING_First-class_Inns']}' , '{row['BATTING_First-class_NO']}' , '{row['BATTING_First-class_Runs']}' , '{row['BATTING_First-class_HS']}' , '{row['BATTING_First-class_Ave']}' , '{row['BATTING_First-class_BF']}' , '{row['BATTING_First-class_SR']}' , '{row['BATTING_First-class_100']}' , '{row['BATTING_First-class_50']}' , '{row['BATTING_First-class_4s']}' , '{row['BATTING_First-class_6s']}' )
        """
        cursor.execute(insert_query)
        conn.commit()


cursor.close()
conn.close()


## Bowling Stats table

In [19]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)

cursor = conn.cursor()
conn.commit()
create_query = '''CREATE TABLE IF NOT EXISTS BowlingStats (
        BowlingStatsID SERIAL PRIMARY KEY,
        FormatId INT REFERENCES Format(FormatID),
        PlayerId INT REFERENCES Player(PlayerId),
        Matches INTEGER,
        Innings varchar(10),
        Balls Varchar(10),
        Runs Varchar(10),
        Wickets VarChar(10),
        BestBowlingForInnings varchar(10),
        BestBowlingForMatch varchar(10),
        Average Varchar(10),
        Economy Varchar(10),
        StrikeRate Varchar(10),
        FourWickets INTEGER,
        FiveWickets INTEGER,
        TenWickets INTEGER,
        CHECK (Matches >= 0),
        CHECK (FourWickets >= 0),
        CHECK (FiveWickets >= 0),
        CHECK (TenWickets >= 0)
    )'''

cursor.execute(create_query)


df['BOWLING_Tests_Mat'] = df['BOWLING_Tests_Mat'].fillna(0).astype(int)
df['BOWLING_Tests_Inns'] = df['BOWLING_Tests_Inns'].fillna(0)
df['BOWLING_Tests_Balls'] = df['BOWLING_Tests_Balls'].fillna(0)
df['BOWLING_Tests_Runs'] = df['BOWLING_Tests_Runs'].fillna(0)
df['BOWLING_Tests_Wkts'] = df['BOWLING_Tests_Wkts'].fillna(0)
df['BOWLING_Tests_BBI'] = df['BOWLING_Tests_BBI'].fillna(0)
df['BOWLING_Tests_BBM'] = df['BOWLING_Tests_BBM'].fillna(0)
df['BOWLING_Tests_Ave'] = df['BOWLING_Tests_Ave'].fillna(0)
df['BOWLING_Tests_Econ'] = df['BOWLING_Tests_Econ'].fillna(0)
df['BOWLING_Tests_SR'] = df['BOWLING_Tests_SR'].fillna(0)
df['BOWLING_Tests_4w'] = df['BOWLING_Tests_4w'].fillna(0).astype(int)
df['BOWLING_Tests_5w'] = df['BOWLING_Tests_5w'].fillna(0).astype(int)
df['BOWLING_Tests_10'] = df['BOWLING_Tests_10'].fillna(0).astype(int)


df['BOWLING_ODIs_Mat'] = df['BOWLING_ODIs_Mat'].fillna(0).astype(int)
df['BOWLING_ODIs_Inns'] = df['BOWLING_ODIs_Inns'].fillna(0)
df['BOWLING_ODIs_Balls'] = df['BOWLING_ODIs_Balls'].fillna(0)
df['BOWLING_ODIs_Runs'] = df['BOWLING_ODIs_Runs'].fillna(0)
df['BOWLING_ODIs_Wkts'] = df['BOWLING_ODIs_Wkts'].fillna(0)
df['BOWLING_ODIs_BBI'] = df['BOWLING_ODIs_BBI'].fillna(0)
df['BOWLING_ODIs_BBM'] = df['BOWLING_ODIs_BBM'].fillna(0)
df['BOWLING_ODIs_Ave'] = df['BOWLING_ODIs_Ave'].fillna(0)
df['BOWLING_ODIs_Econ'] = df['BOWLING_ODIs_Econ'].fillna(0)
df['BOWLING_ODIs_SR'] = df['BOWLING_ODIs_SR'].fillna(0)
df['BOWLING_ODIs_4w'] = df['BOWLING_ODIs_4w'].fillna(0).astype(int)
df['BOWLING_ODIs_5w'] = df['BOWLING_ODIs_5w'].fillna(0).astype(int)
df['BOWLING_ODIs_10'] = df['BOWLING_ODIs_10'].fillna(0).astype(int)




df['BOWLING_T20Is_Mat'] = df['BOWLING_T20Is_Mat'].fillna(0).astype(int)
df['BOWLING_T20Is_Inns'] = df['BOWLING_T20Is_Inns'].fillna(0)
df['BOWLING_T20Is_Balls'] = df['BOWLING_T20Is_Balls'].fillna(0)
df['BOWLING_T20Is_Runs'] = df['BOWLING_T20Is_Runs'].fillna(0)
df['BOWLING_T20Is_Wkts'] = df['BOWLING_T20Is_Wkts'].fillna(0)
df['BOWLING_T20Is_BBI'] = df['BOWLING_T20Is_BBI'].fillna(0)
df['BOWLING_T20Is_BBM'] = df['BOWLING_T20Is_BBM'].fillna(0)
df['BOWLING_T20Is_Ave'] = df['BOWLING_T20Is_Ave'].fillna(0)
df['BOWLING_T20Is_Econ'] = df['BOWLING_T20Is_Econ'].fillna(0)
df['BOWLING_T20Is_SR'] = df['BOWLING_T20Is_SR'].fillna(0)
df['BOWLING_T20Is_4w'] = df['BOWLING_T20Is_4w'].fillna(0).astype(int)
df['BOWLING_T20Is_5w'] = df['BOWLING_T20Is_5w'].fillna(0).astype(int)
df['BOWLING_T20Is_10'] = df['BOWLING_T20Is_10'].fillna(0).astype(int)


df['BOWLING_First-class_Mat'] = df['BOWLING_First-class_Mat'].fillna(0).astype(int)
df['BOWLING_First-class_Inns'] = df['BOWLING_First-class_Inns'].fillna(0)
df['BOWLING_First-class_Balls'] = df['BOWLING_First-class_Balls'].fillna(0)
df['BOWLING_First-class_Runs'] = df['BOWLING_First-class_Runs'].fillna(0)
df['BOWLING_First-class_Wkts'] = df['BOWLING_First-class_Wkts'].fillna(0)
df['BOWLING_First-class_BBI'] = df['BOWLING_First-class_BBI'].fillna(0)
df['BOWLING_First-class_BBM'] = df['BOWLING_First-class_BBM'].fillna(0)
df['BOWLING_First-class_Ave'] = df['BOWLING_First-class_Ave'].fillna(0)
df['BOWLING_First-class_Econ'] = df['BOWLING_First-class_Econ'].fillna(0)
df['BOWLING_First-class_SR'] = df['BOWLING_First-class_SR'].fillna(0)
df['BOWLING_First-class_4w'] = df['BOWLING_First-class_4w'].fillna(0).astype(int)
df['BOWLING_First-class_5w'] = df['BOWLING_First-class_5w'].fillna(0).astype(int)
df['BOWLING_First-class_10'] = df['BOWLING_First-class_10'].fillna(0).astype(int)

for index, row in df.iterrows():
    if row['BOWLING_Tests_Mat'] > 0:

        insert_query = """
    INSERT INTO BowlingStats (FormatId, PlayerId, Matches, Innings, Balls, Runs, Wickets, BestBowlingForInnings, BestBowlingForMatch, Average, Economy, StrikeRate, FourWickets, FiveWickets, TenWickets)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
        values = (
        formats_dict['TEST'],
        players_dict[row['Full name']],
        row['BOWLING_Tests_Mat'],
        row['BOWLING_Tests_Inns'],
        row['BOWLING_Tests_Balls'],
        row['BOWLING_Tests_Runs'],
        row['BOWLING_Tests_Wkts'],
        row['BOWLING_Tests_BBI'],
        row['BOWLING_Tests_BBM'],
        row['BOWLING_Tests_Ave'],
        row['BOWLING_Tests_Econ'],
        row['BOWLING_Tests_SR'],
        row['BOWLING_Tests_4w'],
        row['BOWLING_Tests_5w'],
        row['BOWLING_Tests_10']
        )

        cursor.execute(insert_query, values)
        conn.commit()
    if row['BOWLING_ODIs_Mat'] > 0:
        insert_query = """
    INSERT INTO BowlingStats (FormatId, PlayerId, Matches, Innings, Balls, Runs, Wickets, BestBowlingForInnings, BestBowlingForMatch, Average, Economy, StrikeRate, FourWickets, FiveWickets, TenWickets)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
        values = (
        formats_dict['ODI'],
        players_dict[row['Full name']],
        row['BOWLING_ODIs_Mat'],
        row['BOWLING_ODIs_Inns'],
        row['BOWLING_ODIs_Balls'],
        row['BOWLING_ODIs_Runs'],
        row['BOWLING_ODIs_Wkts'],
        row['BOWLING_ODIs_BBI'],
        row['BOWLING_ODIs_BBM'],
        row['BOWLING_ODIs_Ave'],
        row['BOWLING_ODIs_Econ'],
        row['BOWLING_ODIs_SR'],
        row['BOWLING_ODIs_4w'],
        row['BOWLING_ODIs_5w'],
        row['BOWLING_ODIs_10']
        )

        cursor.execute(insert_query, values)
        conn.commit()
    if row['BOWLING_T20Is_Mat'] > 0:
        insert_query = """
    INSERT INTO BowlingStats (FormatId, PlayerId, Matches, Innings, Balls, Runs, Wickets, BestBowlingForInnings, BestBowlingForMatch, Average, Economy, StrikeRate, FourWickets, FiveWickets, TenWickets)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
        values = (
        formats_dict['T20'],
        players_dict[row['Full name']],
        row['BOWLING_T20Is_Mat'],
        row['BOWLING_T20Is_Inns'],
        row['BOWLING_T20Is_Balls'],
        row['BOWLING_T20Is_Runs'],
        row['BOWLING_T20Is_Wkts'],
        row['BOWLING_T20Is_BBI'],
        row['BOWLING_T20Is_BBM'],
        row['BOWLING_T20Is_Ave'],
        row['BOWLING_T20Is_Econ'],
        row['BOWLING_T20Is_SR'],
        row['BOWLING_T20Is_4w'],
        row['BOWLING_T20Is_5w'],
        row['BOWLING_T20Is_10']
        )

        cursor.execute(insert_query, values)
        conn.commit()
    if row['BOWLING_First-class_Mat'] > 0:
        insert_query = """
    INSERT INTO BowlingStats (FormatId, PlayerId, Matches, Innings, Balls, Runs, Wickets, BestBowlingForInnings, BestBowlingForMatch, Average, Economy, StrikeRate, FourWickets, FiveWickets, TenWickets)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
        values = (
        formats_dict['FirstClass'],
        players_dict[row['Full name']],
        row['BOWLING_First-class_Mat'], 
        row['BOWLING_First-class_Inns'], 
        row['BOWLING_First-class_Balls'], 
        row['BOWLING_First-class_Runs'], 
        row['BOWLING_First-class_Wkts'], 
        row['BOWLING_First-class_BBI'], 
        row['BOWLING_First-class_BBM'], 
        row['BOWLING_First-class_Ave'], 
        row['BOWLING_First-class_Econ'], 
        row['BOWLING_First-class_SR'], 
        row['BOWLING_First-class_4w'], 
        row['BOWLING_First-class_5w'], 
        row['BOWLING_First-class_10']
        )
        cursor.execute(insert_query, values)
        conn.commit()

cursor.close()
conn.close()



## Fielding Stats table

In [20]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)

cursor = conn.cursor()
conn.commit()
create_query = '''CREATE TABLE IF NOT EXISTS FieldingStats (
        FieldingStatsId SERIAL PRIMARY KEY,
        FormatId INT REFERENCES Format(FormatID),
        PlayerId INT REFERENCES Player(PlayerId),
        CatchesTaken INT, 
        Stumpings INT,
        RunOuts INT,
        RunsSaved INT,
        CHECK (CatchesTaken >= 0),
        CHECK (Stumpings >= 0),
        CHECK (RunOuts >= 0),
        CHECK (RunsSaved >= 0)      
    )'''

cursor.execute(create_query)

df['BATTING_Tests_Ct'] = df['BATTING_Tests_Ct'].fillna(0).astype(int)
df['BATTING_Tests_St'] = df['BATTING_Tests_St'].fillna(0).astype(int)
df['Test_Run_Outs'] = df['Test_Run_Outs'].fillna(0).astype(int)
df['Test_Fielding_Runs_Saved'] = df['Test_Fielding_Runs_Saved'].fillna(0).astype(int)

df['BATTING_ODIs_Ct'] = df['BATTING_ODIs_Ct'].fillna(0).astype(int)
df['BATTING_ODIs_St'] = df['BATTING_ODIs_St'].fillna(0).astype(int)
df['ODI_Run_Outs'] = df['ODI_Run_Outs'].fillna(0).astype(int)
df['ODI_Fielding_Runs_Saved'] = df['ODI_Fielding_Runs_Saved'].fillna(0).astype(int)

df['BATTING_T20Is_Ct'] = df['BATTING_T20Is_Ct'].fillna(0).astype(int)
df['BATTING_T20Is_St'] = df['BATTING_T20Is_St'].fillna(0).astype(int)
df['T20_Run_Outs'] = df['T20_Run_Outs'].fillna(0).astype(int)
df['T20_Fielding_Runs_Saved'] = df['T20_Fielding_Runs_Saved'].fillna(0).astype(int)


for index, row in df.iterrows():
    if row['BATTING_Tests_Mat'] > 0:
        insert_query = f"""
        INSERT INTO FieldingStats(FormatId, PlayerId, CatchesTaken, Stumpings, RunOuts, RunsSaved)
        VALUES('{formats_dict['TEST']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_Tests_Ct']}' , '{row['BATTING_Tests_St']}' , '{row['Test_Run_Outs']}' , '{row['Test_Fielding_Runs_Saved']}')
        """
        cursor.execute(insert_query)
        conn.commit()

    if row['BATTING_ODIs_Mat'] > 0:
        insert_query = f"""
        INSERT INTO FieldingStats(FormatId, PlayerId, CatchesTaken, Stumpings, RunOuts, RunsSaved)
        VALUES('{formats_dict['ODI']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_ODIs_Ct']}' , '{row['BATTING_ODIs_St']}' , '{row['ODI_Run_Outs']}' , '{row['ODI_Fielding_Runs_Saved']}')
        """
        cursor.execute(insert_query)
        conn.commit()

    if row['BATTING_T20Is_Mat'] > 0:
        insert_query = f"""
        INSERT INTO FieldingStats(FormatId, PlayerId, CatchesTaken, Stumpings, RunOuts, RunsSaved)
        VALUES('{formats_dict['T20']}' , '{players_dict[row['Full name']]}' , '{row['BATTING_T20Is_Ct']}' , '{row['BATTING_T20Is_St']}' , '{row['T20_Run_Outs']}' , '{row['T20_Fielding_Runs_Saved']}')
        """
        cursor.execute(insert_query)
        conn.commit()

cursor.close()
conn.close()


## Player Rankings table

In [21]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)

cursor = conn.cursor()
conn.commit()
create_query = '''CREATE TABLE IF NOT EXISTS PlayerRankings (
        PlayerRankingId SERIAL PRIMARY KEY,
        PlayerId INT REFERENCES Player(PlayerId),
        BattingRank_ODI INT,
        BowlingRank_ODI INT,
        AllRounderRank_ODI INT,
        BattingRank_T20 INT,
        BowlingRank_T20 INT,
        AllRounderRank_T20 INT,
        BattingRank_TEST INT,
        BowlingRank_TEST INT,
        AllRounderRank_TEST INT,
        CHECK (BattingRank_ODI >= 0),
        CHECK (BowlingRank_ODI >= 0),
        CHECK (AllRounderRank_ODI >= 0),
        CHECK (BattingRank_T20 >= 0),
        CHECK (BowlingRank_T20 >= 0),
        CHECK (AllRounderRank_T20 >= 0),
        CHECK (BattingRank_TEST >= 0),
        CHECK (BowlingRank_TEST >= 0),
        CHECK (AllRounderRank_TEST >= 0)
    )'''

cursor.execute(create_query)

df['BattingRank_ODI'] = df['BattingRank_ODI'].fillna(0).astype(int)
df['BowlingRank_ODI'] = df['BowlingRank_ODI'].fillna(0).astype(int)
df['AllRounderRank_ODI'] = df['AllRounderRank_ODI'].fillna(0).astype(int)

df['BattingRank_T20'] = df['BattingRank_T20'].fillna(0).astype(int)
df['BowlingRank_T20'] = df['BowlingRank_T20'].fillna(0).astype(int)
df['AllRounderRank_T20'] = df['AllRounderRank_T20'].fillna(0).astype(int)

df['BattingRank_Test'] = df['BattingRank_Test'].fillna(0).astype(int)
df['BowlingRank_Test'] = df['BowlingRank_Test'].fillna(0).astype(int)
df['AllRounderRank_Test'] = df['AllRounderRank_Test'].fillna(0).astype(int)


for index, row in df.iterrows():
    if row['BATTING_Tests_Mat'] > 0:
        insert_query = f"""
        INSERT INTO PlayerRankings(PlayerId, BattingRank_ODI, BowlingRank_ODI, AllRounderRank_ODI, BattingRank_T20, BowlingRank_T20, AllRounderRank_T20, BattingRank_TEST, BowlingRank_TEST, AllRounderRank_TEST)
        VALUES('{players_dict[row['Full name']]}' , '{row['BattingRank_ODI']}' , '{row['BowlingRank_ODI']}' , '{row['AllRounderRank_ODI']}' , '{row['BattingRank_T20']}' , '{row['BowlingRank_T20']}' , '{row['AllRounderRank_T20']}' , '{row['BattingRank_Test']}' , '{row['BowlingRank_Test']}' , '{row['AllRounderRank_Test']}')
        """
        cursor.execute(insert_query)
        conn.commit()

cursor.close()
conn.close()


## Player Contract table

In [22]:
conn = psycopg2.connect(
    host="localhost",
    database="ESPNCRICINFO",
    user="postgres",
    password="2001"
)

cursor = conn.cursor()
conn.commit()
create_query = '''CREATE TABLE IF NOT EXISTS PlayerContract (
        PlayerContractId SERIAL PRIMARY KEY,
        PlayerId INT REFERENCES Player(PlayerId),
        ContractValue INT NOT NULL CHECK (ContractValue > 0),
        ContractType CHAR(1) NOT NULL CHECK (ContractType IN ('A', 'B', 'C', 'D')),
        ContractDuration INT NOT NULL CHECK (ContractDuration > 0),
        Endorsements INT NOT NULL CHECK (Endorsements >= 0)
        )'''

cursor.execute(create_query)

df['ContractValue'] = df['ContractValue'].fillna(0).astype(int)
df['ContractDuration'] = df['ContractDuration'].fillna(0).astype(int)
df['Endorsements'] = df['Endorsements'].fillna(0).astype(int)


for index, row in df.iterrows():
    if row['BATTING_Tests_Mat'] > 0:
        insert_query = f"""
                INSERT INTO PlayerContract(PlayerId, ContractValue, ContractType, ContractDuration, Endorsements)
                VALUES('{players_dict[row['Full name']]}' , '{row['ContractValue']}' , '{row['ContractType']}' , '{row['ContractDuration']}' , '{row['Endorsements']}')
                """
        cursor.execute(insert_query)
        conn.commit()
        
cursor.close()
conn.close() 
